<a href="https://colab.research.google.com/github/pthinh14/triplet-loss/blob/master/triplet_loss_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
# %pip install tensorflow==2.0.0
#!pip install tensorflow-gpu --upgrade
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
import tensorflow.keras.applications.inception_v3 as kai
import tensorflow.keras.layers as kl
import tensorflow.keras.models as km
import tensorflow.keras.callbacks as KC
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt

## required for semi-hard triplet loss:
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import dtypes

import os
import numpy as np
from itertools import islice
import random
from sklearn.preprocessing import LabelEncoder

num_classes_per_batch = 4
num_images_per_class = 8

base_path= './drive/My Drive/IKEA/'
IMG_TYPES = ['.png', '.jpg']
IMG_SIZE = 128

le = LabelEncoder()


Requirement already up-to-date: tensorflow-gpu in /usr/local/lib/python3.6/dist-packages (2.0.0)


In [0]:
!pip install -q -U tensorflow>=1.7
import tensorflow
tensorflow.__version__

'2.0.0'

In [0]:
def load_data(path=base_path):
    directories = os.listdir(path)
    img = {}
    for directory in directories:
        sub_dirs = os.listdir(base_path + directory)
        for sub_dir in sub_dirs:
            cur_path = os.path.join(base_path + directory, sub_dir)
            _, _, filenames = next(os.walk(cur_path))
            for filename in filenames:
                if os.path.splitext(filename)[1] in IMG_TYPES:
                    if directory +'/'+ sub_dir not in img:
                        img[directory +'/'+ sub_dir] = [os.path.join(base_path  + directory +'/'+ sub_dir, filename)]
                    else:
                        img[directory +'/'+ sub_dir].append(os.path.join(base_path + directory +'/'+ sub_dir, filename))

    label_encode = le.fit_transform(list(img.keys()))
    return img, label_encode

In [0]:
def triplet_loss(y_true, y_pred):
    embeddings = K.cast(y_pred, 'float32')
    
    embeddings = y_pred
    anchor_positive = embeddings[:num_images_per_class]
    negative = embeddings[num_images_per_class:]

    # Compute pairwise distance between all of anchor-positive
    dot_product = K.dot(anchor_positive, K.transpose(anchor_positive))
    square = K.square(anchor_positive)
    a_p_distance = K.reshape(K.sum(square, axis=1), (-1,1)) - 2.*dot_product  + K.sum(K.transpose(square), axis=0) + 1e-6
    a_p_distance = K.maximum(a_p_distance, 0.0) ## Numerical stability

    # Compute distance between anchor and negative
    dot_product_2 = K.dot(anchor_positive, K.transpose(negative))
    negative_square = K.square(negative)
    a_n_distance = K.reshape(K.sum(square, axis=1), (-1,1)) - 2.*dot_product_2  + K.sum(K.transpose(negative_square), axis=0)  + 1e-6
    a_n_distance = K.maximum(a_n_distance, 0.0) ## Numerical stability
    
    hard_negative = K.reshape(K.min(a_n_distance, axis=1), (-1, 1))
    
    distance = (a_p_distance - hard_negative + 0.2)
    loss = K.mean(K.maximum(distance, 0.0))/(2.)
            
    return loss
    # print(y_true)
    # print(y_pred)
    # y_pred = K.l2_normalize(y_pred,axis=1)
    # batch = 2
    # ref1 = y_pred[0:1,:]
    # pos1 = y_pred[1:num_images_per_class,:]
    # neg1 = y_pred[num_images_per_class:,:]
    # print(ref1)
    # print(pos1)
    # print(neg1)
    # dis_pos = K.sum(K.square(ref1 - pos1), axis=1, keepdims=True)
    # dis_neg = K.sum(K.square(ref1 - neg1), axis=1, keepdims=True)
    # dis_pos = K.sqrt(dis_pos)
    # dis_neg = K.sqrt(dis_neg)
    # a1 = 0.6
    # d1 = K.maximum(0.0,dis_pos-dis_neg+a1)
    # return K.mean(d1)


def build_model(image_shape=(IMG_SIZE, IMG_SIZE, 3), embedding_length=128, trainable=True):
    backbone = kai.InceptionV3(input_shape=image_shape, include_top=False)
    backbone.trainable = trainable
    x = kl.GlobalMaxPooling2D()(backbone.output)
    x = kl.Dense(embedding_length * 2)(x)
    x = kl.Dense(embedding_length)(x)
    embedding = kl.Dense(embedding_length, name='embedding')(x)
    model = km.Model(inputs=[backbone.input], outputs=[embedding])
    # opt = tf.train.AdamOptimizer(0.0001)
    model.compile(loss=triplet_loss, optimizer='adam')
    # model.summary()
    return model



In [0]:
def tfdata_generator(images, labels, is_training, batch_size=128):
    '''Construct a data generator using tf.Dataset'''
    num_labels = len(images)
    def preprocess_fn(filename, label):
        '''A transformation function to preprocess raw data
        into trainable input. '''
        image_string = tf.io.read_file(filename)

        #Don't use tf.image.decode_image, or the output shape will be undefined
        image = tf.image.decode_png(image_string, channels=3)

        #This will convert to float values in [0, 1]
        image = tf.image.convert_image_dtype(image, tf.float32)

        resized_image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
        return resized_image, label


    def generator():
        while True:
            # Sample the labels that will compose the batch
            chosen_labels = random.sample(range(num_labels),
                                        num_classes_per_batch)
            for label in chosen_labels:
                for _ in range(num_images_per_class):
                    # yield images[label][np.random.choice(range(num_images_per_class))]
                    yield label_encode[label]

    datasets = []
    for i in range(len(img)):
        filenames = img[le.inverse_transform([i])[0]]
        labels = [i for _ in range(len(filenames))]
        dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
        datasets.append(dataset)
    
    # dataset = [tf.data.Dataset.from_tensor_slices((images, label_encode))]

    choice_dataset = tf.data.Dataset.from_generator(generator, tf.int64)
    # for d in choice_dataset:
    #     print(d)
    dataset = tf.data.experimental.choose_from_datasets(datasets, choice_dataset)
    # for image_number, label in dataset.batch(4).take(20):
    #     print("images {} from classes {}".format(image_number.numpy(), label.numpy()))
    dataset = dataset.map(preprocess_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size, drop_remainder=True if is_training else False)

    dataset = dataset.repeat()
    dataset = dataset.prefetch(1)
    return dataset

In [6]:

# try:
#     device_name = os.environ["COLAB_TPU_ADDR"]
#     TPU_ADDRESS = "grpc://" + device_name
#     print("Found TPU at: {}".format(TPU_ADDRESS))
# except KeyError:
#     print("TPU not found")

print("hello world")

# data = DataLoader()
model = build_model(trainable=True)
STEPS_PER_EPOCH = 10
batch_size = num_classes_per_batch*num_images_per_class

img, label_encode = load_data()

inc = iter(img.items())
splitting_point = int(len(label_encode)*0.8)
x_train = dict(islice(inc, splitting_point))
x_test = dict(inc)
y_train = label_encode[:splitting_point]
y_test = label_encode[splitting_point:]
training_set = tfdata_generator(x_train, y_train, is_training=True)
validation_set = tfdata_generator(x_test, y_test, is_training=True)

checkpt = KC.ModelCheckpoint('./weights.{epoch:02d}-{loss:.2f}.hdf5',
                             monitor='loss',
                             verbose=0,
                             save_best_only=False, 
                             mode='auto',
                             period=1)
history = model.fit(training_set,
                    steps_per_epoch=STEPS_PER_EPOCH,
                    epochs=20,
                    validation_data=None,
                    validation_steps=None,
                    callbacks=[checkpt],
                    class_weight=None,
                    max_queue_size=10,
                    workers=1,
                    use_multiprocessing=False,
                    shuffle=False)

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


hello world
Train for 10 steps
Epoch 1/20
 5/10 [==============>...............] - ETA: 30s - loss: 62.7685

KeyboardInterrupt: ignored

In [0]:
dataset = [tf.data.Dataset.from_tensor_slices((images, label_encode))]

choice_dataset = tf.data.Dataset.from_generator(generator, tf.int64)
dataset = tf.data.experimental.choose_from_datasets(dataset, choice_dataset)
iterator_helper = dataset.make_iterable_iterator()
with tf.Session() as sess:
    filename_temp = iterator_helper.get_next()
    print(sess.run(filename_temp))
    filename_temp = iterator_helper.get_next()
    print(sess.run(filename_temp))
